# fMRI Machine Learning using Bayesian Regularised Regression

The notebook below demonstrates a Machine learning approach to the analysis of the  structural MRI data. The data provided by the Autism Brain Imaging Data Exchange II (ABIBEII) study from Di Martino et al (2017).The study is international collabortion towards the study of Austism. Specifically, by genereating large samples of structural MRI data from multiple international reseacrh sites. The data analysed below is the lightly preprocessed data from  Bethlehem (2020). Here having been downloaded and stored into the associated Datasets github of this notebooks author (see, the repositories https://github.com/HPCurtis?tab=repositories.

The ABIBE dataset here contains a range of features that can be categorised broadly into three seperate categories. The first of which are simple identifiers (site, subject). The second are phenotype features (age, sex, diagnosis of autism present). Thirdly, the rest of the data set are 4 set of structural brain features (suface Area, volume, cortical thickness and gyrification)(tagged as fsArea, fsVol, fsCT and fsLGI in the dataset). Each of the four features containing 360 variables resulting in 1440 feaautres in total. Each of the these360 features represents a region of interest specified in the Human Connectome Project Glasser et al. (2016).

In [11]:
# Import neccesary python packages.
from cmdstanpy import CmdStanModel, write_stan_json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
import arviz as az
import os

## Define useful functions for the analysis below 

In [12]:
# Define pythonised Prediction mean squared error function for Bayesian regularized regression models.
# Based on R function from https://github.com/sara-vanerp/bayesreg/blob/master/R/pmse_lm.R
def pmse(object, ytest):
    draws = object.draws_pd(vars = 'y_test')
    draws = draws.T
    postmeans = draws[0].mean()
    pmse = np.mean((postmeans-ytest)**2)
    return pmse, draws[0] 

${ \frac{1}{N} \Sigma^N_{i=1} (y^{gen}_i - y_i)^2 }$

# Data cleaning

In [13]:
# Read ABIDEII dataset from github using pandas.
d =  pd.read_csv('https://raw.githubusercontent.com/HPCurtis/Datasets/main/abide2.csv')

In [14]:
# View the top three rows of dataset.
d.head(3)

,Unnamed: 0,site,subject,age,sex,group,age_resid,fsArea_L_V1_ROI,fsArea_L_MST_ROI,fsArea_L_V6_ROI,...,fsCT_R_p47r_ROI,fsCT_R_TGv_ROI,fsCT_R_MBelt_ROI,fsCT_R_LBelt_ROI,fsCT_R_A4_ROI,fsCT_R_STSva_ROI,fsCT_R_TE1m_ROI,fsCT_R_PI_ROI,fsCT_R_a32pr_ROI,fsCT_R_p24_ROI
0,0,ABIDEII-KKI_1,29293,8.893151,2.0,1.0,13.642852,2750.0,306.0,354.0,...,3.362,2.827,2.777,2.526,3.202,3.024,3.354,2.629,2.699,3.179
1,1,ABIDEII-OHSU_1,28997,12.000000,2.0,1.0,16.081732,2836.0,186.0,354.0,...,2.809,3.539,2.944,2.769,3.530,3.079,3.282,2.670,2.746,3.324
2,2,ABIDEII-GU_1,28845,8.390000,1.0,2.0,12.866264,3394.0,223.0,373.0,...,2.435,3.321,2.799,2.388,3.148,3.125,3.116,2.891,2.940,3.232


In [15]:
# Extract 
y = d.age
features = d.filter(like='fs')

In [16]:
# Split data into training and test sets using sklearn train_test_split.
X_train, X_test, y_train, y_test = train_test_split(features, y, train_size = 0.8)

# Standardise the y_test data so they are on the same scale so predciton error is on the 
# equivalent scale
y_test = scale(np.asarray(y_test))

# Create python dictionary to passs the data of the anlaysis to
# the prerequisite .JSON file below
dataDic = {'N_train': len(X_train),
           'N_test': len(X_test),
           'p': np.shape(X_train)[1],
           'X_train': X_train,
           'X_test': X_test,
           'y_train': y_train
           }

# Write JSON file for data to be passed to Stan.
write_stan_json(path = 'RegRegABIDE.json' , data = dataDic)

# Bayesian LASSO regression

In [29]:
# Compile stan model into C++.
modelL = CmdStanModel(stan_file = 'RegLasso.stan', cpp_options={'STAN_THREADS':'true'})

14:54:54 - cmdstanpy - INFO - compiling stan file /home/harrison/Desktop/gitHubRepos/ML_port/fMRI/RegLasso.stan to exe file /home/harrison/Desktop/gitHubRepos/ML_port/fMRI/RegLasso
14:55:14 - cmdstanpy - INFO - compiled model executable: /home/harrison/Desktop/gitHubRepos/ML_port/fMRI/RegLasso
14:55:14 - cmdstanpy - WARNING - Stan compiler has produced 1 warnings:
14:55:14 - cmdstanpy - WARNING - 
--- Translating Stan model to C++ code ---
bin/stanc  --o=/home/harrison/Desktop/gitHubRepos/ML_port/fMRI/RegLasso.hpp /home/harrison/Desktop/gitHubRepos/ML_port/fMRI/RegLasso.stan
Warning in '/home/harrison/Desktop/gitHubRepos/ML_port/fMRI/RegLasso.stan', line 50, column 1: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc

--- Compiling, linking C++ code ---
/home/harrison/anaconda3/envs/ML_env/bi

In [32]:
# Use varaitional infernce to calculate init values to speed up HMC sampling below. 
vbfitL = modelL.variational(data ='RegRegABIDE.json', require_converged = False, seed = 1, grad_samples = 10)
vbL_vars = vbfitL.stan_variables()

14:58:42 - cmdstanpy - INFO - Chain [1] start processing
15:00:27 - cmdstanpy - INFO - Chain [1] done processing
15:00:27 - cmdstanpy - WARNING - The algorithm may not have converged.
Proceeding because require_converged is set to False


In [ ]:
# fit the Stan model to the data.
fitL = modelL.sample(data ='RegRegABIDE.json', chains = 4, inits = vbL_vars, iter_sampling = 1000, iter_warmup = 500);

15:00:27 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

In [ ]:
az.summary(fitL)
LassoDrawsDf = fitL.draws_pd()

# Bayesian Ridge Regression analysis

In [17]:
# Compile stan model into C++.
modelR = CmdStanModel(stan_file = 'RegReg.stan', cpp_options={'STAN_THREADS':'true'})

14:44:58 - cmdstanpy - INFO - compiling stan file /home/harrison/Desktop/gitHubRepos/ML_port/fMRI/RegReg.stan to exe file /home/harrison/Desktop/gitHubRepos/ML_port/fMRI/RegReg
14:45:17 - cmdstanpy - INFO - compiled model executable: /home/harrison/Desktop/gitHubRepos/ML_port/fMRI/RegReg
14:45:17 - cmdstanpy - WARNING - Stan compiler has produced 1 warnings:
14:45:17 - cmdstanpy - WARNING - 
--- Translating Stan model to C++ code ---
bin/stanc  --o=/home/harrison/Desktop/gitHubRepos/ML_port/fMRI/RegReg.hpp /home/harrison/Desktop/gitHubRepos/ML_port/fMRI/RegReg.stan
Warning in '/home/harrison/Desktop/gitHubRepos/ML_port/fMRI/RegReg.stan', line 54, column 1: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc

--- Compiling, linking C++ code ---
/home/harrison/anaconda3/envs/ML_env/bin/x86_64-con

In [18]:
# Use varaitional infernce to calculate init values to speed up HMC sampling below. 
vbfitR = modelR.variational(data ='RegRegABIDE.json', require_converged = False, seed = 1, grad_samples = 10)
vbR_vars = vbfitR.stan_variables()

14:45:17 - cmdstanpy - INFO - Chain [1] start processing
14:47:11 - cmdstanpy - INFO - Chain [1] done processing
14:47:11 - cmdstanpy - WARNING - The algorithm may not have converged.
Proceeding because require_converged is set to False


In [ ]:
# fit the Stan model to the data.
fitR = modelR.sample(data ='RegRegABIDE.json', chains = 4, inits = vbR_vars, iter_sampling = 1000, iter_warmup = 500);

In [21]:
az.summary(fitR)
RidgeDrawsDf = fitR.draws_pd()

,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,mu,sigma2,beta[1],...,y_test[192],y_test[193],y_test[194],y_test[195],y_test[196],y_test[197],y_test[198],y_test[199],y_test[200],y_test[201]
0,5435.94,0.998358,0.047989,6.0,127.0,0.0,-4720.42,-0.011029,0.223000,-0.010036,...,-1.087670,2.02977,-0.012876,-0.727844,1.300440,2.09717,-0.392504,-0.738327,-0.332318,0.043353
1,5442.47,0.873523,0.047989,7.0,127.0,0.0,-4710.05,0.010525,0.222001,0.000343,...,-0.329745,1.61867,-0.578166,-0.603608,0.689908,1.89219,-0.345908,-0.303403,-0.745769,0.205405
2,5432.94,0.940511,0.047989,7.0,127.0,0.0,-4678.85,0.011783,0.212446,-0.021616,...,-0.014595,1.45626,-1.703580,-0.210036,0.135441,2.34865,-0.343946,-0.270916,-0.082922,0.295281
3,5411.90,0.795983,0.047989,6.0,127.0,0.0,-4699.52,-0.006152,0.224174,0.016366,...,-0.879587,2.40952,0.115888,-0.854251,0.654837,2.02926,-0.697933,-1.002760,-0.094188,-1.510250
4,5484.12,0.906310,0.047989,7.0,127.0,0.0,-4715.38,0.002719,0.217375,-0.029023,...,-0.822948,1.79609,-0.699386,-1.254450,0.766367,2.16545,-0.414403,-0.526175,1.000290,0.765211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,5310.14,0.869759,0.061672,6.0,63.0,0.0,-4561.99,-0.009532,0.231494,0.023197,...,-0.882979,1.07527,0.245690,0.653686,1.026740,1.59879,0.073992,-2.084450,-1.169800,-0.133682
3996,5323.86,0.994131,0.061672,6.0,63.0,0.0,-4571.96,-0.016769,0.218662,0.024060,...,-0.851036,2.09401,-0.830166,-0.304265,1.873780,1.80966,-0.221676,-0.718294,0.001126,0.560246
3997,5355.86,0.878169,0.061672,6.0,63.0,0.0,-4605.25,-0.002254,0.221169,0.016280,...,-1.073030,1.23086,-0.543395,-1.389700,0.549993,2.38571,-0.594476,-0.468773,-0.069867,0.014201
3998,5447.37,0.951582,0.061672,6.0,63.0,0.0,-4665.41,0.012305,0.220888,-0.017642,...,-0.179388,1.70345,-1.047590,-0.126710,0.900509,2.62505,-0.574544,-1.778280,0.096722,1.047200


In [25]:
# Compute the point estimate of the prediction error
mse, draws = pmse(fitR, y_test)
mse

1.0000035152332198

 ## Plot Predictions against test set

In [ ]:
sns.jointplot(x = y_test, y = draws );

# References 

Bethlehem, R. A., Seidlitz, J., Romero-Garcia, R., Trakoshis, S., Dumas, G., & Lombardo, M. V. (2020). A normative modelling approach reveals age-atypical cortical thickness in a subgroup of males with autism spectrum disorder. Communications biology, 3(1), 486.

Di Martino, A., O’connor, D., Chen, B., Alaerts, K., Anderson, J. S., Assaf, M., ... & Milham, M. P. (2017). Enhancing studies of the connectome in autism using the autism brain imaging data exchange II. Scientific data, 4(1), 1-15.

Glasser, M. F., Coalson, T. S., Robinson, E. C., Hacker, C. D., Harwell, J., Yacoub, E., ... & Van Essen, D. C. (2016). A multi-modal parcellation of human cerebral cortex. Nature, 536(7615), 171-178.